In [2]:
import os
import numpy as np
import cv2

# Define sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define ReLU activation function
def relu(x):
    return np.maximum(0, x)

# Define derivative of sigmoid
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Define derivative of ReLU
def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# Define paths to train and test folders
train_folder = r'C:\Users\diyam\Documents\datase\Train'
test_folder = r'C:\Users\diyam\Documents\mrlEyes_2018_01\Prepared_Data\Test'

classes = ['open', 'closed']

X_train = []
y_train = []
X_test = []
y_test = []

img_size = (92, 112)

for i, class_name in enumerate(classes):
    class_path = os.path.join(train_folder, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
        X_train.append(img)
        y_train.append(i)

    class_path = os.path.join(test_folder, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
        X_test.append(img)
        y_test.append(i)

X_train = np.array(X_train, dtype=np.uint8)
y_train = np.array(y_train)
X_test = np.array(X_test, dtype=np.uint8)
y_test = np.array(y_test)

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = np.eye(2)[y_train]
y_test = np.eye(2)[y_test]

learning_rate = 0.001  # Define the learning rate

# Update Conv2D layer to correctly handle input channel dimension
class Conv2D:
    def __init__(self, filters, kernel_size, activation='relu'):
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = activation
        self.weights = None
        self.bias = np.zeros(filters)

    def forward(self, inputs):
        self.inputs = inputs
        if self.weights is None:
            input_channels = inputs.shape[-1]
            # Adjust the initialization of weights to match input_channels
            self.weights = np.random.randn(self.kernel_size[0], self.kernel_size[1], input_channels, self.filters)
        # Perform the convolution operation using numpy
        conv_output = np.zeros((inputs.shape[0] - self.kernel_size[0] + 1,
                                inputs.shape[1] - self.kernel_size[1] + 1,
                                self.filters))
        for k in range(self.filters):
            for i in range(conv_output.shape[0]):
                for j in range(conv_output.shape[1]):
                    # Adjust the broadcasting of inputs and weights
                    conv_output[i, j, k] = np.sum(inputs[i:i+self.kernel_size[0], j:j+self.kernel_size[1], :] *
                                                   self.weights[:, :, :, k]) + self.bias[k]
        if self.activation == 'relu':
            return relu(conv_output)
        elif self.activation == 'sigmoid':
            return sigmoid(conv_output)

    def backward(self, d_out):
        d_input = np.zeros_like(self.inputs)
        d_weights = np.zeros_like(self.weights)
        d_bias = np.zeros_like(self.bias)
        if self.activation == 'relu':
            activation_derivative = relu_derivative
        elif self.activation == 'sigmoid':
            activation_derivative = sigmoid_derivative
        for k in range(self.filters):
            for i in range(d_out.shape[0]):
                for j in range(d_out.shape[1]):
                    d_input[i:i+self.kernel_size[0], j:j+self.kernel_size[1], :] += d_out[i, j, k] * self.weights[:, :, :, k]
                    d_weights[:, :, :, k] += d_out[i, j, k] * self.inputs[i:i+self.kernel_size[0], j:j+self.kernel_size[1], :]
                    d_bias[k] += d_out[i, j, k]
        return d_input, d_weights, d_bias



# Define custom MaxPooling2D Layer
class MaxPooling2DLayer:
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, inputs):
        self.inputs = inputs
        pooled_output = np.zeros((inputs.shape[0] // self.pool_size[0],
                                  inputs.shape[1] // self.pool_size[1],
                                  inputs.shape[2]))
        for i in range(0, inputs.shape[0], self.pool_size[0]):
            for j in range(0, inputs.shape[1], self.pool_size[1]):
                for k in range(inputs.shape[2]):
                    pooled_output[i//self.pool_size[0], j//self.pool_size[1], k] = np.max(inputs[i:i+self.pool_size[0], j:j+self.pool_size[1], k])
        return pooled_output

    def backward(self, d_out):
        d_input = np.zeros_like(self.inputs)
        for i in range(d_out.shape[0]):
            for j in range(d_out.shape[1]):
                for k in range(self.inputs.shape[2]):
                    window = self.inputs[i*self.pool_size[0]:i*self.pool_size[0]+self.pool_size[0],
                                          j*self.pool_size[1]:j*self.pool_size[1]+self.pool_size[1], k]
                    max_value = np.max(window)
                    d_input[i*self.pool_size[0]:i*self.pool_size[0]+self.pool_size[0],
                            j*self.pool_size[1]:j*self.pool_size[1]+self.pool_size[1], k] = (window == max_value) * d_out[i, j, k]
        return d_input

# Define custom Dropout Layer
class DropoutLayer:
    def __init__(self, rate):
        self.rate = rate

    def forward(self, inputs, training=True):
        if training:
            self.mask = np.random.rand(*inputs.shape) < self.rate
            return np.where(self.mask, 0, inputs)
        else:
            return inputs

    def backward(self, d_out):
        return np.where(self.mask, 0, d_out)

# Define custom Flatten Layer
class Flatten:
    def forward(self, inputs):
        self.input_shape = inputs.shape
        return inputs.flatten().reshape((inputs.shape[0], -1))

    def backward(self, d_out):
        return d_out.reshape(self.input_shape)

# Define custom Dense Layer
class Dense:
    def __init__(self, units, activation='relu'):
        self.units = units
        self.activation = activation
        self.weights = np.random.randn(units)
        self.bias = np.zeros(units)

    def forward(self, inputs):
        self.inputs = inputs
        dense_output = np.dot(inputs, self.weights) + self.bias
        if self.activation == 'relu':
            return relu(dense_output)
        elif self.activation == 'sigmoid':
            return sigmoid(dense_output)

    def backward(self, d_out):
        d_input = np.dot(d_out, self.weights.T)
        d_weights = np.dot(self.inputs.T, d_out)
        d_bias = np.sum(d_out, axis=0)
        return d_input, d_weights, d_bias

# Define custom Softmax Layer
class Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        return probabilities

    def backward(self, y_true, y_pred):
        batch_size = len(y_true)
        d_loss = y_pred - y_true
        return d_loss / batch_size

# Define custom Sequential model
class Sequential:
    def __init__(self, layers):
        self.layers = layers

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, inputs, training=True):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs

    def backward(self, d_out):
        for layer in reversed(self.layers):
            d_out = layer.backward(d_out)
        return d_out

    def predict(self, inputs):
        return self.forward(inputs, training=False)

    def train_step(self, X, y):
        # Forward pass
        logits = self.forward(X)
        loss = self.loss_fn(y, logits)
        
        # Backward pass
        grad = self.loss_fn.backward(y, logits)
        self.backward(grad)
        
        return loss

    def fit(self, X_train, y_train, epochs, batch_size, validation_data=None):
        for epoch in range(epochs):
            epoch_loss = 0
            for i in range(0, len(X_train), batch_size):
                X_batch = X_train[i:i+batch_size]
                y_batch = y_train[i:i+batch_size]
                loss = self.train_step(X_batch, y_batch)
                epoch_loss += loss
            epoch_loss /= (len(X_train) / batch_size)
            print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss}')
            if validation_data:
                val_loss = self.evaluate(validation_data[0], validation_data[1])
                print(f'Validation Loss: {val_loss}')

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        loss = self.loss_fn(y_test, y_pred)
        return loss

    def compile(self, loss):
        self.loss_fn = loss

# Define custom Categorical Crossentropy Loss
class CategoricalCrossentropyLoss:
    def __call__(self, y_true, y_pred):
        # Avoid division by zero
        epsilon = 1e-15
        # Clip values to prevent NaNs in log
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        # Compute cross-entropy loss
        loss = -np.sum(y_true * np.log(y_pred)) / len(y_true)
        return loss

    def backward(self, y_true, y_pred):
        # Avoid division by zero
        epsilon = 1e-15
        # Clip values to prevent NaNs in log
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        # Compute gradient of loss with respect to y_pred
        grad = -y_true / y_pred
        return grad

# Define custom Accuracy metric
class Accuracy:
    def __call__(self, y_true, y_pred):
        return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))

# Create the model
model = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2DLayer(pool_size=(2, 2)),
    DropoutLayer(rate=0.25),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2DLayer(pool_size=(2, 2)),
    DropoutLayer(rate=0.25),
    Flatten(),
    Dense(units=128, activation='relu'),
    DropoutLayer(rate=0.5),
    Dense(units=2, activation='softmax')
])

# Compile the model
model.compile(loss=CategoricalCrossentropyLoss())

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


ValueError: operands could not be broadcast together with shapes (3,3,92,1) (3,3,1) 